## Convolution-LSTM Neural Network for music generation

In [1]:
#Imports
import tensorflow as tf
import pretty_midi
import pandas as pd
import numpy as np
import glob
import pathlib
from random import randint
import collections

### Load the models


In [37]:
model_p_C = tf.keras.models.load_model('./models/pitch-model-Classic.keras')
model_p_P = tf.keras.models.load_model('./models/pitch-model-POP.keras')

model_i_C = tf.keras.models.load_model('./models/intervall-model-Classic.keras')
model_i_P = tf.keras.models.load_model('./models/intervall-model-POP.keras')

model_p_M = tf.keras.models.load_model('./models/pitch-model-Multi.keras')
#model_i_M = tf.keras.models.load_model('./models/intervall-model-Multi.keras')


In [33]:
#initial sequence
data_dir_c = pathlib.Path(r'../maestro-v2.0.0')
data_dir_p = pathlib.Path(r'../midi_dataset_pop')

filenames_c = glob.glob(str(data_dir_c/'**/*.mid*'))
filenames_p = glob.glob(str(data_dir_p/'**/*.mid*'))

# random music
file = randint(1,50)

sample_file_c = filenames_c[file]
sample_file_p = filenames_p[file]

pm_c = pretty_midi.PrettyMIDI(sample_file_c)
pm_p = pretty_midi.PrettyMIDI(sample_file_p)

instrument_c = pm_c.instruments[0]
instrument_p = pm_p.instruments[0]

def midi_to_notes(midi_file: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)

    # Sort the notes by start time
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start
    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

pitch_notes_c = midi_to_notes(sample_file_c)
pitch_notes_p = midi_to_notes(sample_file_p)

c:\Users\cavas\anaconda3\Lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [ ]:
# intervall encoding
def calc_intervallo(pitch1:int,pitch2:int):
    semitoni = (pitch2 - pitch1) % 12  # Usa il modulo per gestire le ottave

    return int(abs(semitoni))

def calc_diff(pitch1:int,pitch2:int)->int:
    ottave = (pitch2 - pitch1) / 12
    return int(ottave)

def create_intervall_data(df):
    notes = [] 
    rate = 16
    prev_note = df[0:1]
    for _, row in df.iterrows():
        intervall = calc_intervallo(prev_note['pitch'],row['pitch'])
        difference = calc_diff(prev_note['pitch'],row['pitch'])
        duration = int(row['duration']*rate)
        step = int(row['step']*rate)
        notes.append([intervall,difference,step,duration])
    return pd.DataFrame(notes,columns=['intervallo','diff','step','duration'])


#!!!! usare questi per i modelli ad intervalli
intervall_notes_c = create_intervall_data(pitch_notes_c[:10])
intervall_notes_p = create_intervall_data(pitch_notes_p[:10])

initial_pitch_c = pitch_notes_c[:1]['pitch']
initial_pitch_c = pitch_notes_p[:1]['pitch']

C:\Users\cavas\AppData\Local\Temp\ipykernel_3236\577691530.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(abs(semitoni))
C:\Users\cavas\AppData\Local\Temp\ipykernel_3236\577691530.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(ottave)


In [65]:
def generate_notes(model,seed,length):
    musica = seed.copy()
    l_seed = len(musica)

    for _ in range(length - l_seed):
        print(len(musica))
        input_seq = musica[-l_seed:]
        # Add a batch dimension to the input
        input_seq = np.array(input_seq) 
        input_seq = np.expand_dims(input_seq, axis=0)
        note = model.predict(input_seq)

        intervall = np.argmax(note['intervall'])
        diff = np.argmax(note['diff'])
        step = np.argmax(note['step'])
        duration = np.argmax(note['duration'])
        print(len(musica))

        nuova_nota = pd.DataFrame([[intervall, diff, step, duration]], 
                                 columns=musica.columns) # Crea un DataFrame per la nuova nota
        musica = pd.concat([musica, nuova_nota], ignore_index=True)

    return musica


gen_notes = generate_notes(model_i_C, intervall_notes_c,30) # controllare il dataset e il modello se vanno bene

10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
10
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
11
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
12
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
13
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
14
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
15
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
16
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
17
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
18
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
19
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
20
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
21
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
22
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
23
24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
24
25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
25
26
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
26
27
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
27
28
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
28
29
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
29


In [71]:
# Dato il pitch iniziale e gli intervalli calcola il pitch di ogni nota
def calc_pitch(initial_pitch,intervall,diff):
    diff_d = diff - 4 # Decode the diff
    pitch_hop=0
    if (diff_d > 1):
        pitch_hop = (diff_d -2) * 12
    if (diff_d < -1):
        pitch_hop = (diff_d +2) * 12
    pitch_finale = initial_pitch + intervall + pitch_hop
    return abs(pitch_finale)

def decode_sequence(initial_pitch, intervalls, diffs):
    results = []
    pitch = initial_pitch

    # Correct way to iterate through the *values* of Pandas Series or lists:
    for intervallo, diff in zip(intervalls, diffs): # Assuming intervalls and diffs are the same length
        decoded_note_pitch = calc_pitch(pitch, intervallo, diff)
        pitch = decoded_note_pitch
        results.append(pitch)

    return results

seq = decode_sequence(initial_pitch_c,gen_notes['intervallo'],gen_notes['diff'])
seq_array = np.array(seq, dtype=np.int32)
notes = gen_notes.drop(columns=['intervallo','diff'])
notes = notes.assign( pitch=seq_array)
notes

,step,duration,pitch
0,0,1,45
1,1,1,23
2,1,1,1
3,0,0,12
4,4,0,12
5,2,0,10
6,2,0,14
7,2,28,0
8,0,1,22
9,2,0,2


In [72]:
# Convert a list of notes to a midi file
def notes_to_midi(notes: pd.DataFrame, out_file: str, instrument_name: str,
                  velocity: int = 100) -> pretty_midi.PrettyMIDI:

    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

    prev_start = 0
    for i, note in notes.iterrows():
        start = float(prev_start + note['step'])
        end = float(start + note['duration'])
        
        note = pretty_midi.Note(velocity=velocity, pitch=int(note['pitch']),
                                start=start, end=end)
        instrument.notes.append(note)
        prev_start = start

    pm.instruments.append(instrument)
    pm.write(out_file)
    return pm

example_file = '../generate/intervallClassic.midi'
example_pm = notes_to_midi(
    notes, out_file=example_file, instrument_name='Acoustic Grand Piano')# metti gen_notes se il modello è il pitch e notes se ...

In [73]:
import pygame

def riproduci_midi(file_midi):
    try:
        pygame.mixer.init()
        pygame.mixer.music.load(file_midi)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10) 

    except pygame.error as e:
        print(f"Errore durante la riproduzione del file MIDI: {e}")
    finally:
        pygame.mixer.quit()  


file_midi = example_file
riproduci_midi(file_midi)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 